In [0]:
pip install openai

In [0]:
%run ./docs_generation_helpers

In [0]:
from openai import AzureOpenAI
import re
import requests
import json
import os
from urllib.parse import urljoin
import base64 

In [0]:
endpoint = "https://azweuglobkt360.openai.azure.com/openai/deployments/DevGPT4omini/chat/completions?api-version=2024-08-01-preview"

api_version = "2024-08-01-preview"
api_key = "9i2y1gaIL9IftXdqAeMTJWIvxOZCM78z0jBXHhwV8luVlgp9uMz2JQQJ99AKAC5RqLJXJ3w3AAABACOGIG11"
client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    azure_endpoint = endpoint
    )

In [0]:
folder = "/Workspace/dnst-data-platform/datalake/000_landing/sharepoint/stage_agora_culinair_van_hout_5525957_v0"
all_notebook_details = get_notebook_info_from_folder(folder)

In [0]:
for notebook_name, details in all_notebook_details.items():
    cleaned_content = clean_notebook_content(details['content'])

In [0]:
user_prompt = cleaned_content

In [0]:
# Refined system_prompt
system_prompt = """
You are an expert technical writer for Databricks. Your goal is to generate high-quality, professional documentation for notebooks.

### STYLE GUIDELINES:
- Use clean headers (##, ###) and horizontal rules (---) to separate sections.
- Use emojis in headers (e.g., 📦, 🔗, ⚙️) to match the modern Databricks style.
- Use bold text for parameters and backticks for function names.
- If the source code contains "TODO" or developer questions (e.g., "CAN WE RENAME?"), ignore the question and document only the current logic.

### CONTENT STRUCTURE:
1. Title and Description: A clear # H1 title followed by a brief summary.
2. Configuration/Parameters: A list of key variables or matching rules.
3. Logical Steps: Break down the code into meaningful steps with explanations.
4. Technical Details: Use LaTeX for math and ```python code blocks for the core logic.

### IMPORTANT:
Return the output as a JSON object: {"md": "YOUR_MARKDOWN_HERE"}. 
The Markdown inside must be formatted EXACTLY like a high-end Databricks landing page, with proper spacing, clear bullet points, and distinct sections.
"""

In [0]:
docs = call_open_ai_api(system_prompt, user_prompt)

In [0]:
data = json.loads(docs[0])
print(data['md']) # This will render the \n as actual new lines

In [0]:
import json

# 1. Parse the string into a dictionary
# 'docs[0]' is the raw string from your API response
parsed_data = json.loads(docs[0])

# 2. Now you can access the 'md' key safely
pretty_markdown = parsed_data['md']

# 3. Print it to see the clean, rendered Markdown
print(pretty_markdown)

# 📥 Download and Stage Excel Files from SharePoint

This notebook is designed to download Excel files from a specified SharePoint folder, convert them to CSV format, and write the resulting files to a designated landing area. Additionally, it captures metadata about the processed files for further analysis.

---

## ⚙️ Configuration/Parameters

- **target_catalog**: `d_dnst_000_landing`  
  The catalog where metadata will be written.
- **target_schema**: `metadata`  
  The schema for the metadata table.
- **target_table**: `Agora_Culinair_Van_Hout_5525957_V0_metadata`  
  The name of the metadata table.
- **landing_root**: `/Volumes/d_dnst_000_landing/volumes/sellout_data/5525957___Agora_Culinair_Van_Hout___V0`  
  The root directory for landing files.
- **sharepoint_folder**: `Collection sell-out data/5525957 - Agora Culinair Van Hout - V0`  
  The SharePoint folder containing the files.

---

## 📄 Logical Steps

1. **Authentication**:  
   Retrieve an access token to authenticate with the SharePoint API using client credentials.

2. **Get Site & Drive IDs**:  
   Fetch the site and drive IDs necessary for accessing files in SharePoint.

3. **List Files Recursively**:  
   Recursively list all files in the specified SharePoint folder while skipping any folders defined in the configuration.

4. **Download, Convert to CSV, Write to Landing**:  
   For each file, download it, convert the relevant sheets to CSV format, and save them to the landing directory. Collect metadata for each processed file.

---

## 🔍 Technical Details

### Authentication Function

```python
def get_token():
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": "https://graph.microsoft.com/.default"
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]
```

### List Files Function

```python
def list_files(folder_path):
    enc = urllib.parse.quote(folder_path, safe="")
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{enc}:/children"
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    items = r.json()["value"]
    files = []
    for it in items:
        name = it.get("name")
        path = f"{folder_path}/{name}"
        if "file" in it and fnmatch.fnmatch(name, config["file_pattern"]):
            files.append(path)
        elif "folder" in it:
            if any(skip.lower() in name.lower() for skip in config["folder_name_skip"]):
                print(f"⏩ Skipping folder: {name}")
                continue
            files.extend(list_files(path))
    return files
```

### Download and Stage Function

```python
def download_and_stage_excel_files():
    files_to_download = list_files(sharepoint_folder)
    metadata_rows = []
    run_start_time = time.strftime("%Y-%m-%d %H:%M:%S")  # Common timestamp for all rows

    for sp_path in files_to_download:
        try:
            enc = urllib.parse.quote(sp_path, safe="")
            url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root:/{enc}:/content"
            r = requests.get(url, headers=headers, stream=True)
            r.raise_for_status()

            buf = BytesIO(r.content)
            xls = pd.ExcelFile(buf)
            filename_base = os.path.splitext(os.path.basename(sp_path))[0].replace(" ", "_").replace("-", "_")

            if config["sheet_name"]:
                sheets_to_process = [config["sheet_name"]] if config["sheet_name"] in xls.sheet_names else []
            else:
                sheets_to_process = [s for s in xls.sheet_names if s not in (config["sheet_name_skip"] or [])]

            for sheet in sheets_to_process:
                df = pd.read_excel(xls, sheet_name=sheet)
                out_filename = f"{filename_base}_{sheet}.csv"
                relative_path = os.path.dirname(sp_path.replace(sharepoint_folder, "").lstrip("/"))
                out_dir = f"{landing_root}/{relative_path}"
                dbutils.fs.mkdirs(out_dir)
                out_path = f"{out_dir}/{out_filename}"

                csv_data = df.to_csv(index=False)
                dbutils.fs.put(out_path, csv_data, overwrite=True)

                metadata_rows.append(Row(
                    source_path=sp_path,
                    sheet_name=sheet,
                    output_path=out_path,
                    row_count=len(df),
                    column_count=len(df.columns),
                    timestamp=time.strftime("%Y-%m-%d %H:%M:%S"),
                    runstarttime=run_start_time
                ))

                print(f"✅ Saved {out_path}")

        except Exception as e:
            print(f"❌ Error processing {sp_path}: {e}")

    df_metadata = spark.createDataFrame(metadata_rows)

    df_metadata = (
        df_metadata
        .withColumn("row_count", F.col("row_count").cast(IntegerType()))
        .withColumn("column_count", F.col("column_count").cast(IntegerType()))
        .withColumn("timestamp", F.to_timestamp("timestamp"))
        .withColumn("runstarttime", F.to_timestamp("runstarttime"))
    )
    return df_metadata
```

---

### Execution

To execute the download and staging process, call the function:

```python
fn_flowbase = download_and_stage_excel_files
```

This will initiate the process of downloading Excel files, converting them to CSV, and saving the metadata.

In [0]:
def create_markdown_notebook(notebook_path, markdown_content):
    """
    Creates a new Databricks notebook containing the provided markdown.
    """
    # 1. Get Context (same logic as your export function)
    try:
        host = os.environ.get("DATABRICKS_HOST")
        if not host:
            host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
        token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
    except Exception as e:
        print(f"Error retrieving Databricks context: {e}")
        return

    # 2. Prepare the content
    # We wrap the markdown in MAGIC %md so Databricks renders it as a cell
    full_content = f"# Databricks notebook source\n# MAGIC %md\n{markdown_content}"
    
    # Content must be base64 encoded for the API
    encoded_content = base64.b64encode(full_content.encode("utf-8")).decode("utf-8")

    # 3. Prepare API Request
    url = urljoin(host, "/api/2.0/workspace/import")
    headers = {"Authorization": f"Bearer {token}"}
    
    payload = {
        "path": notebook_path,
        "format": "SOURCE",
        "language": "PYTHON",
        "content": encoded_content,
        "overwrite": True  # Change to False if you don't want to overwrite existing docs
    }

    # 4. Execute
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        print(f"✅ Successfully created notebook at: {notebook_path}")
    except Exception as e:
        print(f"❌ Failed to create notebook: {e}")



In [0]:
# --- Example Usage ---
new_path = "/Workspace/Users/remi.tossut@vandemoortele.com/databricks-dnst/dnst-data-platform/documentation/test_generation"
create_markdown_notebook(new_path, pretty_markdown)